# US alert Function
Chelsy Xia
 April 1st 2020



This functions selects the US cases of COVID-19 on a state scale and define state's alert level.\
Inputs: US cases from MongoDB, State names\
Outputs: slope,three day average acceleration rate, alert level.\
Delta-infect is calculated by the difference between most recent date and the day before.
Three day average acceleration rate is calculated from the three day average of acceleration.\
Alert level is denfined according to :\
Red: cases >= 100, acceleration > 0, slope > 0\
Yellow: cases >=100, acceleration <= 0 , slope >= 0\
Green: cases <100 or slope < 0 


In [5]:
! pip install pymongo

In [48]:
import pymongo
from pymongo import MongoClient
import numpy as np


class Mongo_connector(object):
    def __init__(self, server_ip, user, pw, table):
        self.mongo_client = MongoClient(server_ip + ':27017',
                            username=user,
                            password=pw,
                            authSource='COVID19-DB',
                            authMechanism='SCRAM-SHA-256')
        self.data = self.mongo_client["COVID19-DB"][table]
        
    def get_states_data(self,countryname, statename, lastn):
        docs = self.data.find({"$and": [{"Country/Region": {"$eq": countryname}},
                                {"Province/State": {"$eq": statename}}]}).sort("Date", pymongo.DESCENDING).limit(lastn)
        number = []
        dateList = []
        for doc in docs:
            update_date = doc.get("Date")
            date_obj = update_date.date()
            date_str = date_obj.isoformat()
            dateList.append(date_str)
            number.append([doc.get("Confirmed"), doc.get("Death")])
        dateList.reverse()
        number.reverse()
        number = np.array(number, dtype = 'float')
        return dateList, number
        
        
        


if __name__ == "__main__":
    con = Mongo_connector('3.101.18.8', 'analyst', 'grmds', 'CDC-TimeSeries')
    date, number = con.get_states_data("US","Iowa",5)
    

In [49]:
number

array([[29.,  0.],
       [44.,  0.],
       [45.,  0.],
       [68.,  0.],
       [90.,  0.]])

In [50]:
date

['2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22']

In [51]:
def task(date, number):
    most_recent_infect = number[-1][0]
    most_recent_d = number[-1][1]
    delta_infect = number[-1][0] - number[-2][0]
    three_day_acc = ((number[-1][0] - 2*number[-2][0] + number[-3][0])/2 + 
                     (number[-2][0] - 2*number[-3][0] + number[-4][0])/2 + 
                     (number[-3][0] - 2*number[-4][0] + number[-5][0])/2)/3
    if most_recent_infect >100 and three_day_acc <= 0 and delta_infect >=0:
         alertlevel = "Yellow"
    if most_recent_infect >100 and three_day_acc > 0 and delta_infect >0:
        alertlevel = "Red"     
    else:
        alertlevel="Green"
    return most_recent_infect, most_recent_d, delta_infect, three_day_acc,alertlevel

In [52]:
task(date,number)

(90.0, 0.0, 22.0, 1.1666666666666667, 'Green')